In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
import random
import numpy as np
from collections import Counter

In [4]:
lemmatizer = WordNetLemmatizer()
hm_lines = 100000


In [6]:
# def create_lexicon(pos,neg):
#     lexmizer = []
#     for fi in [pos,neg]:
#         with open(fi,'r') as f:
#             content = f.readlines()
#             for line in content[:hm_lines]:
#                 total_words = word_tokenize(line.lower())
#                 lexmizer +=list(total_words)
                
#             lexmizer  = [lemmatizer.lemmatize(i) for i in lexmizer]
            
#         l2 = []
#         w_count = Counter(lexmizer)
#         for w in w_count:
#             if 2000 > w_count[w] >50 :
#                 l2.append(w)
#     print(f"lexicon_size : {len(l2)}")
#     return l2

# def create_sample_handler(sample , lexicon , classification):
#     featureset = []
#     with open(sample , 'r') as f:
#         content = f.readlines()
#         for line in content:
#             total_word = word_tokenize(line.lower())
#             leximize = [lemmatizer.lemmatize(i) for i in total_word]
#             features = np.zeros(len(lexicon))
#             for w in leximize:
#                 if w.lower() in lexicon:
#                     index_value = lexicon.index(w.lower())
#                     features[index_value] += 1
                    
#             features = list(features)
#             featureset.append([features , classification])
#     print(f"{sample} : {len(featureset)}")
#     return featureset

# def create_data_save(pos , neg):
#     lexicon = create_lexicon(pos,neg)
#     features = []
#     features += create_sample_handler(pos, lexicon , [1,0])
#     features +=create_sample_handler(neg,lexicon , [0,1])
    
#     lexicon_file = open("lexicon.pickle" , 'wb')
#     pickle.dump(lexicon , lexicon_file)
    
#     features_file = open("features.pickle" , 'wb')
#     pickle.dump(features , features_file)

# create_data_save('pos.txt' , 'neg.txt')

lexicon_size : 431
pos.txt : 5331
neg.txt : 5331


In [8]:
# def create_train_test_data(test_size = 0.1):
#     features = open('features.pickle' , 'rb')
#     features = pickle.load(features)
    
#     features = np.array(features)
    
#     testing_size = int(len(features) * test_size)
    
#     train_x = list(features[:,0][:-testing_size])
#     train_y = list(features[:,1][:-testing_size])
    
#     test_x = list(features[:,0][-testing_size:])
#     test_y = list(features[:,1][-testing_size:])
    
#     lexi = open('dataprocessed.pickle' , 'wb')
#     pickle.dump([train_x , train_y , test_x , test_y] , lexi)
#     print(len(train_x) , len(train_y))
#     print(len(test_x) , len(test_y))

#     print('done.. use "dataprocessed.pickle" for further training purpose')
# create_train_test_data()

9596 9596
1066 1066
done.. use "dataprocessed.pickle" for further training purpose


In [5]:
import pickle
import random
import numpy as np
fi =  open('dataprocessed.pickle' , 'rb')
train_x , train_y , test_x , test_y = pickle.load(fi)

import torch
import torch.nn as nn
import torchvision
# from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# train_x , train_y , test_x , test_y = np.array(train_x) , np.array( train_y) , np.array( test_x ), np.array( test_y)

# train_data = torch.utils.data.TensorDataset(torch.from_numpy(train_x) , torch.from_numpy(train_y))
# test_data = torch.utils.data.TensorDataset(torch.from_numpy(test_x) , torch.from_numpy(test_y))

# train_loader = torch.utils.data.DataLoader(train_data , shuffle=True , batch_size=100)
# test_loader = torch.utils.data.DataLoader(train_data , shuffle=True , batch_size=100)

# The size of lexicon = 431
# ip_size = train_x[0]
ip_size = 431
op_size = 2

class NN(nn.Module):
    def __init__(self , ip_size , op_size , embedding_dim = 600 , hidden_size = 1000, dropout=0.3 , n_layers = 2):
        super().__init__()
        self.ip_szze = ip_size
        self.op_size  =op_size
        self.hidden_size = hidden_size
        self.drop_prob = dropout
        self.n_layers = 2
        
        
        self.embedding = nn.Embedding(ip_size , embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim ,self.hidden_size, n_layers  , dropout=self.drop_prob , batch_first=True)
        self.dropout =  nn.Dropout(0.3)
        self.fcc1 = nn.Linear(self.hidden_size  , int(self.hidden_size / 2))
        self.fcc2 = nn.Linear(int(self.hidden_size / 2) , op_size)
        self.relu = nn.Softmax()
        
    def forward(self , input ,hidden):
        input = input.t()
        batch_size = input.size(0)        
        input = input.long()        
        embedded = self.embedding(input) 
        lstm_out ,hidden = self.lstm1(embedded , hidden)  
        
        out = self.dropout(lstm_out[-1])
        out = out.view(-1 , self.hidden_size)
        
        out = self.fcc1(out)

        x = self.relu(out)
        x = self.fcc2(x)  
        return x ,hidden
    
    def init_hidden(self , batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers , batch_size , self.hidden_size).zero_() ,
                  weight.new(self.n_layers, batch_size, self.hidden_size).zero_())
        return hidden
    
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
model = NN(ip_size , op_size)
model.to(device)
loss_function = nn.MSELoss()
optim = torch.optim.Adam(model.parameters() , lr=0.01)

In [2]:
# epochs = 5
# for epoch in range(epochs):
#     h = model.init_hidden(batch_size=ip_size)

#     for i , batch in enumerate(train_loader):
#         model.zero_grad

#         text , label = batch
#         text = text.to(device)
#         text = text.float()
#         label = label.float()
#         label = label.to(device)
#         h = tuple([each.data for each in h])

#         output , h = model(text, h)

#         loss = loss_function(output, label)
#         loss.backward()
#         optim.step()
#     if(epoch+1) % 2 == 0:
#         print(f'Epoch- {epoch + 1}  : loss- {loss.squeeze()}')

In [1]:
import torch.nn as nn

In [6]:
model

NN(
  (embedding): Embedding(431, 600)
  (lstm1): LSTM(600, 1000, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3)
  (fcc1): Linear(in_features=1000, out_features=500, bias=True)
  (fcc2): Linear(in_features=500, out_features=2, bias=True)
  (relu): Softmax()
)

In [55]:
# string = label = None
# size = 30
# total_correct = total_count = 0
# prect_len = 0
# for i , (string , labels) in enumerate(test_loader):

#     string = string.to(device)
#     label = labels.to(device)
#     label = label.long()
#     string = string.float()

#     output = model(string)

#     _ , predicted = torch.max(output.data , 1)

#     predicted = predicted.cpu()
#     label = label.cpu()
#     prect_len += len(predicted)
#     total_count +=label.size(0)
#     total_correct += (predicted == label[0][i]).sum().item()
    
# print(f"Accuray {100 * total_correct / total_count} , correct:{total_correct} , total={total_count} :: Trained with epoch '5'")
# print(prect_len)

RuntimeError: Expected hidden[0] size (2, 431, 1400), got (100, 700)

In [8]:
model_loaded = torch.load("lstmmodel.pt")

In [9]:
model_loaded

NN(
  (embedding): Embedding(431, 600)
  (lstm1): LSTM(600, 1000, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3)
  (fcc1): Linear(in_features=1000, out_features=500, bias=True)
  (fcc2): Linear(in_features=500, out_features=2, bias=True)
  (relu): Softmax()
)

In [10]:

def read_lexicon():
    global lexicon 

    lexi = open("lexicon.pickle" , 'rb')
    lexicon = pickle.load(lexi)
    
    return lexicon


In [14]:
ip_size = 431
def user_test(sample , lexicon):
    
    c_w = word_tokenize(sample.lower())
    c_w = [lemmatizer.lemmatize(i) for i in c_w]
    samples = np.zeros(len(lexicon))
    for  word in c_w:
        if word.lower() in lexicon:
            index_value = lexicon.index(word.lower())
            samples[index_value] += 1
            
    samples = list(samples)  
    output = []
    output.append(samples)
    output = np.array(output)
    return output

def predict_data(data ):
    lexicon = read_lexicon()
    data  = user_test(data , lexicon)
    data = torch.from_numpy(data)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data = data.to(device)
    data = data.float()
    hidden = model_loaded.init_hidden(batch_size=ip_size)
    predict , hidden = model_loaded(data , hidden)
    print(predict[0])
    prediction = torch.max(predict , 1) [1]
    result = prediction.cpu()
    result  = result.numpy()
    if int(result) == 0:
        print("--------------------------")
        print("Positive")
        print("--------------------------")
    elif int(result) == 1:
        print("--------------------------")
        print("Negative")
        print("--------------------------")
    else:
        print(int(result) , "Got this value")
    

In [23]:
data = input()
result =  predict_data(data)


 Good but not satisfied with the speed


tensor([0.7065, 2.4867], device='cuda:0', grad_fn=<SelectBackward>)
--------------------------
Negative
--------------------------


/usr/lib/python3/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
